In [17]:
import pandas as pd
import numpy as np
import sklearn as skl
import os.path as osp
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from numpy import sqrt
import statsmodels.formula.api as smf

In [58]:
df = pd.read_excel('nba23-22.xlsx',index_col='Index')
df.rename(columns={col: col.replace('%', 'pct') for col in df.columns}, inplace=True)
df['difference']=df['total']-df['total_opp']
df['won']=df['total']>df['total_opp']

def add_target(group):
    group["target"] = group["difference"].shift(-1)
    return group

df = df.groupby("team", group_keys=False).apply(add_target)

df_cleaned = df.dropna(subset=['target'])

print(df_cleaned)


        mp  mp.1  fg  fga  fgpct  3p  3pa  3ppct  ft  fta  ...  ortg_max_opp  \
Index                                                      ...                 
1      240   240  33   96  0.344   9   35  0.257  14   16  ...           202   
2      240   240  38   84  0.452   5   28  0.179  13   23  ...           102   
3      240   240  39   79  0.494  14   28  0.500  16   21  ...           223   
4      240   240  35   78  0.449   8   25  0.320  17   20  ...           164   
5      240   240  41   80  0.513   5   18  0.278  22   27  ...           134   
...    ...   ...  ..  ...    ...  ..  ...    ...  ..  ...  ...           ...   
5216   240   240  44   86  0.512   9   30  0.300   5    8  ...           129   
5217   240   240  48   91  0.527  13   32  0.406  14   17  ...           205   
5218   240   240  43   84  0.512   9   31  0.290  17   20  ...           173   
5220   240   240  43   83  0.518  15   31  0.484  14   22  ...           145   
5221   240   240  32   84  0.381  12   4

C:\Users\Jose Chua\AppData\Local\Temp\ipykernel_15668\1963121061.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("team", group_keys=False).apply(add_target)


In [48]:
df

,mp,mp.1,fg,fga,fgpct,3p,3pa,3ppct,ft,fta,...,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,difference,won,target
Index,,,,,,,,,,,,,,,,,,,,,
1,240,240,33,96,0.344,9,35,0.257,14,16,...,202,104,DEN,94,1,2023,2023-06-12,-5,False,-13.0
2,240,240,38,84,0.452,5,28,0.179,13,23,...,102,109,MIA,89,0,2023,2023-06-12,5,True,13.0
3,240,240,39,79,0.494,14,28,0.500,16,21,...,223,127,MIA,95,1,2023,2023-06-09,13,True,15.0
4,240,240,35,78,0.449,8,25,0.320,17,20,...,164,114,DEN,108,0,2023,2023-06-09,-13,False,-15.0
5,240,240,41,80,0.513,5,18,0.278,22,27,...,134,129,MIA,94,1,2023,2023-06-07,15,True,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5246,240,240,36,87,0.414,14,37,0.378,12,17,...,158,109,DEN,110,0,2022,2021-10-20,-12,False,NaN
5247,240,240,41,93,0.441,14,39,0.359,25,30,...,209,115,LAL,114,1,2022,2021-10-19,7,True,NaN
5248,240,240,45,95,0.474,15,42,0.357,9,19,...,174,108,GSW,121,0,2022,2021-10-19,-7,False,NaN


In [59]:
test_data = pd.read_excel('nba_test.xlsx')
test_data.rename(columns={col: col.replace('%', 'pct') for col in test_data.columns}, inplace=True)
test_data['difference']=test_data['total']-test_data['total_opp']
test_data['won']=test_data['total']>test_data['total_opp']

def add_target(group):
    group["target"] = group["difference"].shift(-1)
    return group

test_data = test_data.groupby("team", group_keys=False).apply(add_target)
test_data = test_data.dropna(subset=['target'])

C:\Users\Jose Chua\AppData\Local\Temp\ipykernel_15668\3527534221.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_data = test_data.groupby("team", group_keys=False).apply(add_target)


In [60]:
train_data = pd.read_excel('nba_train.xlsx')
train_data.rename(columns={col: col.replace('%', 'pct') for col in train_data.columns}, inplace=True)
train_data['difference']= train_data['total']-train_data['total_opp']
train_data['won']=train_data['total']>train_data['total_opp']

def add_target(group):
    group["target"] = group["difference"].shift(-1)
    return group

train_data = train_data.groupby("team", group_keys=False).apply(add_target)
train_data = test_data.dropna(subset=['target'])

C:\Users\Jose Chua\AppData\Local\Temp\ipykernel_15668\20370005.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_data = train_data.groupby("team", group_keys=False).apply(add_target)


In [61]:
del df['mp.1']
del df['mp_opp.1']
del df['index_opp']

del test_data['mp.1']
del test_data['mp_opp.1']
del test_data['index_opp']

del train_data['mp.1']
del train_data['mp_opp.1']
del train_data['index_opp']

In [62]:
test_data[test_data["team"] == "TOR"]

,Index,mp,fg,fga,fgpct,3p,3pa,3ppct,ft,fta,...,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,difference,won,target
175,176,240,38,87,0.437,11,31,0.355,18,36,...,179,122,CHI,109,0,2023,2023-04-12,-4,False,16.0
199,200,240,48,95,0.505,11,33,0.333,14,16,...,172,127,MIL,105,0,2023,2023-04-09,16,True,-19.0
220,221,240,40,93,0.430,9,31,0.290,13,17,...,233,118,BOS,121,1,2023,2023-04-07,-19,False,-4.0
260,261,240,37,90,0.411,6,33,0.182,13,17,...,148,104,BOS,97,1,2023,2023-04-05,-4,False,20.0
276,277,240,42,88,0.477,15,33,0.455,21,28,...,166,132,CHO,100,1,2023,2023-04-04,20,True,20.0
312,313,240,56,104,0.538,8,28,0.286,8,8,...,189,140,CHO,108,1,2023,2023-04-02,20,True,-7.0
320,321,240,42,84,0.500,8,26,0.308,18,23,...,177,131,PHI,117,1,2023,2023-03-31,-7,False,14.0
377,378,240,45,93,0.484,8,36,0.222,8,9,...,131,128,MIA,92,0,2023,2023-03-28,14,True,10.0
407,408,240,45,98,0.459,11,31,0.355,13,16,...,242,128,WAS,104,0,2023,2023-03-26,10,True,21.0
427,428,240,41,92,0.446,13,37,0.351,23,29,...,171,129,DET,97,0,2023,2023-03-24,21,True,-4.0


In [89]:
X_train = train_data[['fga',
 'trbpct',
 'usgpct',
 'fg_max',
 'ftpct_max',
 'orb_max',
 'drb_max',
 'blk_max',
 'gmsc_max',
 'drbpct_max',
 'trbpct_max',
 'stlpct_max',
 'tovpct_max',
 'stl_opp',
 'pts_opp',
 'trbpct_opp',
 'astpct_opp',
 'stlpct_opp',
 'usgpct_opp',
 'ft_max_opp',
 'ftpct_max_opp',
 'trb_max_opp',
 'pts_max_opp',
 'gmsc_max_opp',
 'orbpct_max_opp',
 'trbpct_max_opp',
 'astpct_max_opp',
 'stlpct_max_opp',
 'usgpct_max_opp',
 'total_opp']].values

Y_train = train_data['target'].values

In [93]:
X_test = test_data[['fga',
 'trbpct',
 'usgpct',
 'fg_max',
 'ftpct_max',
 'orb_max',
 'drb_max',
 'blk_max',
 'gmsc_max',
 'drbpct_max',
 'trbpct_max',
 'stlpct_max',
 'tovpct_max',
 'stl_opp',
 'pts_opp',
 'trbpct_opp',
 'astpct_opp',
 'stlpct_opp',
 'usgpct_opp',
 'ft_max_opp',
 'ftpct_max_opp',
 'trb_max_opp',
 'pts_max_opp',
 'gmsc_max_opp',
 'orbpct_max_opp',
 'trbpct_max_opp',
 'astpct_max_opp',
 'stlpct_max_opp',
 'usgpct_max_opp',
 'total_opp']].values

Y_test = test_data['target'].values

In [24]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(
  #  X,Y,train_size=0.95,random_state=0)

In [94]:
reg = LinearRegression().fit(X_train, Y_train)

In [95]:
print(reg.predict(X_test)[0:5])

[-1.53685921  0.75010181  4.13038311 -0.20107119  6.42785578]


In [97]:
column_names = ['fga', 'trbpct', 'usgpct', 'fg_max', 'ftpct_max', 'orb_max', 'drb_max', 'blk_max', 
                 'gmsc_max', 'drbpct_max', 'trbpct_max', 'stlpct_max', 'tovpct_max', 'stl_opp', 
                 'pts_opp', 'trbpct_opp', 'astpct_opp', 'stlpct_opp', 'usgpct_opp', 'ft_max_opp', 
                 'ftpct_max_opp', 'trb_max_opp', 'pts_max_opp', 'gmsc_max_opp', 'orbpct_max_opp', 
                 'trbpct_max_opp', 'astpct_max_opp', 'stlpct_max_opp', 'usgpct_max_opp','total_opp']


X_test_df = pd.DataFrame(X_test, columns=column_names)
predictions = reg.predict(X_test)

In [98]:
X_test_df['Actual'] = Y_test
X_test_df['Predicted'] = predictions
new_df = X_test_df
new_df['team']=test_data['team']
new_df['team_opp']=test_data['team_opp']
new_df = new_df.iloc[::2].reset_index(drop=True)

In [99]:
new_df.head(30)

,fga,trbpct,usgpct,fg_max,ftpct_max,orb_max,drb_max,blk_max,gmsc_max,drbpct_max,...,orbpct_max_opp,trbpct_max_opp,astpct_max_opp,stlpct_max_opp,usgpct_max_opp,total_opp,Actual,Predicted,team,team_opp
0,96.0,43.6,100.0,9.0,1.000,5.0,7.0,2.0,15.6,37.6,...,15.5,18.4,57.2,3.5,26.4,94.0,-13.0,-1.536859,MIA,DEN
1,79.0,47.9,100.0,11.0,1.000,2.0,10.0,3.0,24.6,34.9,...,7.9,17.9,33.5,2.2,30.3,95.0,15.0,4.130383,DEN,MIA
2,80.0,63.7,100.0,12.0,1.000,3.0,18.0,3.0,32.6,63.1,...,74.8,45.2,31.0,3.6,95.2,94.0,-3.0,6.427856,DEN,MIA
3,78.0,44.9,100.0,8.0,1.000,3.0,9.0,2.0,19.3,47.3,...,10.7,18.3,40.8,10.8,42.8,108.0,-11.0,0.939033,MIA,DEN
4,96.0,48.9,100.0,13.0,1.000,4.0,9.0,1.0,20.1,100.0,...,7.0,16.5,54.5,2.5,28.8,104.0,19.0,4.439672,MIA,DEN
5,86.0,51.2,100.0,12.0,1.000,3.0,9.0,1.0,25.5,30.7,...,48.1,26.4,24.1,3.3,35.6,84.0,-1.0,1.622198,MIA,BOS
6,78.0,50.0,100.0,9.0,1.000,5.0,9.0,3.0,25.7,32.5,...,14.2,18.9,31.8,5.9,27.1,103.0,13.0,2.688626,BOS,MIA
7,78.0,49.3,100.0,8.0,1.000,3.0,6.0,1.0,18.7,37.9,...,37.9,24.9,42.2,9.5,59.8,110.0,-17.0,2.563013,MIA,BOS
8,84.0,47.0,100.0,14.0,1.000,2.0,10.0,2.0,27.6,26.4,...,99.2,52.6,34.0,4.4,41.4,99.0,-26.0,0.440625,BOS,MIA
9,85.0,51.3,100.0,11.0,1.000,4.0,11.0,3.0,29.8,31.5,...,13.4,21.5,36.1,2.8,28.2,111.0,11.0,1.486561,DEN,LAL


In [106]:
def check_both_positive_or_negative(row):
    return (row['Actual'] > 0 and row['Predicted'] > 0) or (row['Actual'] < 0 and row['Predicted'] < 0)

new_df['Correct Guess'] = new_df.apply(check_both_positive_or_negative, axis=1)

In [107]:
new_df.head(30)

,fga,trbpct,usgpct,fg_max,ftpct_max,orb_max,drb_max,blk_max,gmsc_max,drbpct_max,...,trbpct_max_opp,astpct_max_opp,stlpct_max_opp,usgpct_max_opp,total_opp,Actual,Predicted,team,team_opp,Correct Guess
0,96.0,43.6,100.0,9.0,1.000,5.0,7.0,2.0,15.6,37.6,...,18.4,57.2,3.5,26.4,94.0,-13.0,-1.536859,MIA,DEN,True
1,79.0,47.9,100.0,11.0,1.000,2.0,10.0,3.0,24.6,34.9,...,17.9,33.5,2.2,30.3,95.0,15.0,4.130383,DEN,MIA,True
2,80.0,63.7,100.0,12.0,1.000,3.0,18.0,3.0,32.6,63.1,...,45.2,31.0,3.6,95.2,94.0,-3.0,6.427856,DEN,MIA,False
3,78.0,44.9,100.0,8.0,1.000,3.0,9.0,2.0,19.3,47.3,...,18.3,40.8,10.8,42.8,108.0,-11.0,0.939033,MIA,DEN,False
4,96.0,48.9,100.0,13.0,1.000,4.0,9.0,1.0,20.1,100.0,...,16.5,54.5,2.5,28.8,104.0,19.0,4.439672,MIA,DEN,True
5,86.0,51.2,100.0,12.0,1.000,3.0,9.0,1.0,25.5,30.7,...,26.4,24.1,3.3,35.6,84.0,-1.0,1.622198,MIA,BOS,False
6,78.0,50.0,100.0,9.0,1.000,5.0,9.0,3.0,25.7,32.5,...,18.9,31.8,5.9,27.1,103.0,13.0,2.688626,BOS,MIA,True
7,78.0,49.3,100.0,8.0,1.000,3.0,6.0,1.0,18.7,37.9,...,24.9,42.2,9.5,59.8,110.0,-17.0,2.563013,MIA,BOS,False
8,84.0,47.0,100.0,14.0,1.000,2.0,10.0,2.0,27.6,26.4,...,52.6,34.0,4.4,41.4,99.0,-26.0,0.440625,BOS,MIA,False
9,85.0,51.3,100.0,11.0,1.000,4.0,11.0,3.0,29.8,31.5,...,21.5,36.1,2.8,28.2,111.0,11.0,1.486561,DEN,LAL,True


In [108]:
percentage_true = new_df['Correct Guess'].mean() * 100
print(f"Percentage of True values in 'Correct': {percentage_true:.2f}%")

Percentage of True values in 'Correct': 54.69%


In [100]:
print((reg.predict(X_test)))

[-1.53685921  0.75010181  4.13038311 ...  1.56346576 -2.05426092
 -2.26268836]


In [101]:
model = ols('total ~ fga+ trbpct+ usgpct+ fg_max+ ftpct_max+ orb_max+ drb_max+ blk_max+ gmsc_max+ drbpct_max+ trbpct_max+ stlpct_max+ tovpct_max+ stl_opp+ pts_opp+ trbpct_opp+ astpct_opp+ stlpct_opp+ usgpct_opp+ ft_max_opp+ ftpct_max_opp+ trb_max_opp+ pts_max_opp+ gmsc_max_opp+ orbpct_max_opp+trbpct_max_opp+ astpct_max_opp+ stlpct_max_opp+ usgpct_max_opp+ total_opp',df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  total   R-squared:                       0.547
Model:                            OLS   Adj. R-squared:                  0.545
Method:                 Least Squares   F-statistic:                     233.5
Date:                Sun, 04 Aug 2024   Prob (F-statistic):               0.00
Time:                        03:27:33   Log-Likelihood:                -18628.
No. Observations:                5250   AIC:                         3.731e+04
Df Residuals:                    5222   BIC:                         3.750e+04
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.0153      0.042     -0.

In [103]:
print("R^2:",reg.score(X_train,Y_train))
print("Root Mean Squared Error:",sqrt(
    mean_squared_error(Y_train,reg.predict(X_train))))
print("Mean Absolute Error:",mean_absolute_error(
    Y_train,reg.predict(X_train)))

R^2: 0.03215020918418321
Root Mean Squared Error: 13.818796109295867
Mean Absolute Error: 11.102891971456957


In [109]:
print("R^2:",reg.score(X_test,Y_test))
print("Root Mean Squared Error:",sqrt(
    mean_squared_error(Y_test,reg.predict(X_test))))
print("Mean Absolute Error:",mean_absolute_error(
    Y_test,reg.predict(X_test)))

R^2: 0.03215020918418321
Root Mean Squared Error: 13.818796109295867
Mean Absolute Error: 11.102891971456957
